In [1]:
from diffusion_libs import get_network, DiffusionModel, scale_dataset
from samples_generators import fill_vocabulary_c_v3, convert_back_to_code_c_v3, vocabulary_c_v3
import tensorflow as tf
from tensorflow import keras
import numpy as np
import tensorflow as tf

In [2]:
fill_vocabulary_c_v3()
# sampling
min_signal_rate = 0.02
max_signal_rate = 0.95

# architecture
embedding_dims = 32
embedding_max_frequency = 1000.0
embedding_min_frequency = 1.0

# optimization
batch_size = 16
ema = 0.999
learning_rate = 1e-3

# dictionary related
DICTIONARY_SIZE = len(vocabulary_c_v3)
TOKENS_CAPACITY = 256

widths = [64, 64, 96, 128, 256]
block_depth = 2

lang_base = f"E:\Studies\master_thesis\diffusion_models_checpoints_savespace\\final_checkpoints\simple_c_v3"
model_path = f"{lang_base}\cp-0128\model"

In [3]:
network = get_network(
      TOKENS_CAPACITY, embedding_min_frequency, embedding_max_frequency, 
      embedding_dims, widths=widths, block_depth=block_depth, name="complicated"
  )

model = DiffusionModel(
      TOKENS_CAPACITY, DICTIONARY_SIZE, network, batch_size, max_signal_rate, 
      min_signal_rate, ema, False
  )

model.compile(
    optimizer = keras.optimizers.experimental.Adam(
        learning_rate=learning_rate
    ),
    loss = keras.losses.mean_absolute_error
)

#normalizer
n_w = np.load(f"{lang_base}\\normalizer_weights.npy", allow_pickle=True)
normalizer = keras.layers.Normalization(mean=n_w[0], variance=n_w[1])
normalizer.build((TOKENS_CAPACITY))
model.normalizer = normalizer
model.load_weights(model_path)

### First should generate some sample to see if it still generates something that looks like code

In [4]:
raw, denormalized = model.generate(1,80)
for sample in denormalized:
  scaled = scale_dataset(sample, DICTIONARY_SIZE)
  print(" ".join(convert_back_to_code_c_v3(scaled)).replace(";", ";\n").replace("{", "{\n").replace("}", "}\n"))
  print()

void ID0 ( char ID1 , char ID2 ) {
 int ID3 ;
 int ID4 = STRING + STRING ;
 char ID5 ;
 int ID6 = ID4 / STRING + NUM ;
 char ID9 ( ID6 | STRING + ID5 + STRING ;
 ID3 = ID3 / NUM - ID2 + STRING ;
 ID3 = STRING ;
 printf ( STRING , NUM ) ;
 ID4 = STRING + STRING + NUM ;
 for ( int ID9 = NUM ;
 ID9 , = NUM ;
 ID8 + - ) {
 char ID9 ;
 char ID9 ;
 int ID10 = STRING ;
 }
 return NUM ;
 }
 EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EMPTY EM

In [ ]:
no = 2
scaled = scale_dataset(denormalized[no], DICTIONARY_SIZE)
convs = convert_back_to_code_c_v3(scaled)
n = 5
chunked = [convs[i:i + n] for i in range(0, len(convs), n)]
for symbols in chunked:
  print(" ".join(symbols))

In [ ]:
l = np.around(denormalized[no], 3).tolist()
chunked = [l[i:i + n] for i in range(0, len(l), n)]
for symbols in chunked:
  print(np.around(np.asarray(symbols), 3))